# Анализ базы данных книжного сервиса. SQL

## Цели исследования

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.

Ваша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

## Описание данных

**Таблица books**

**Содержит данные о книгах:**

book_id — идентификатор книги;

author_id — идентификатор автора;

title — название книги;

num_pages — количество страниц;

publication_date — дата публикации книги;

publisher_id — идентификатор издателя.


**Таблица authors**

**Содержит данные об авторах:**

author_id — идентификатор автора;

author — имя автора.


**Таблица publishers**

**Содержит данные об издательствах:**

publisher_id — идентификатор издательства;

publisher — название издательства;


**Таблица ratings**

**Содержит данные о пользовательских оценках книг:**

rating_id — идентификатор оценки;

book_id — идентификатор книги;

username — имя пользователя, оставившего оценку;

rating — оценка книги

.
**Таблица reviews**

**Содержит данные о пользовательских обзорах на книги:**

review_id — идентификатор обзора;

book_id — идентификатор книги;

username — имя пользователя, написавшего обзор;

text — текст обзора.

## Исследуем таблицы

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine
# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
 'pwd': 'Sdf4$2;d-d30pp', # пароль
 'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
 'port': 6432, # порт подключения
 'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
 db_config['pwd'],
 db_config['host'],
 db_config['port'],
 db_config['db'])
# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

**Функия выводящая результаты SQL-запроса**

In [2]:
def sql_result(query):
  return pd.io.sql.read_sql(query, con = engine)

### Запрос к таблице books

In [41]:
query = \
'''

SELECT * 
  FROM books
   LIMIT 5

'''
sql_result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


### Запрос к таблице authors

In [42]:
query = \
'''

SELECT * 
  FROM authors
    LIMIT 5

'''
sql_result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


### Запрос к таблице publishers

In [43]:
query = \
'''

SELECT * 
  FROM publishers
    LIMIT 5

'''
sql_result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


### Запрос к таблице ratings

In [44]:
query = \
'''

SELECT * 
  FROM ratings
   LIMIT 5

'''
sql_result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


### Запрос к таблице reviews

In [15]:
query = \
'''

SELECT * 
  FROM reviews
   LIMIT 5

'''
sql_result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


## Проведем исследование

### Посчитаем количество книг вышедших после 1 января 2000 года

In [45]:
query = \
'''

SELECT COUNT (book_id) as book_cnt
  FROM books
    WHERE publication_date > '2000-01-01'

'''
sql_result(query)

,book_cnt
0,819


### Подсчитаем количество обзоров и среднюю оценку для каждой книги

In [48]:
query = \
'''

SELECT
      books.book_id,
      books.title,
      COUNT (DISTINCT review_id) AS review_cnt,
      AVG(rating) AS avg_rating
  FROM
      books
        LEFT JOIN 
  reviews ON reviews.book_id = books.book_id
  
        LEFT JOIN
  ratings ON ratings.book_id = books.book_id
  
  GROUP BY
    books.book_id 
  ORDER BY
    COUNT(reviews.review_id) DESC 
  
'''
sql_result(query)

,book_id,title,review_cnt,avg_rating
0,948,Twilight (Twilight #1),7,3.662500
1,750,The Hobbit or There and Back Again,6,4.125000
2,673,The Catcher in the Rye,6,3.825581
3,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.414634
4,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.287500
...,...,...,...,...
995,221,Essential Tales and Poems,0,4.000000
996,387,Leonardo's Notebooks,0,4.000000
997,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.000000
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.666667


<div class="alert alert-block alert-info Исправил. Может в книгах есть двух томник, который имеет два id.

### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц — так мы исключим из анализа брошюры

In [49]:
query = \
'''

SELECT
     publisher,
     COUNT (book_id) as book_cnt
  FROM
     books
      LEFT JOIN
  publishers ON publishers.publisher_id=books.publisher_id
  WHERE
      num_pages>50
  GROUP BY
      publisher
  ORDER BY
      book_cnt DESC

'''
sql_result(query)

,publisher,book_cnt
0,Penguin Books,42
1,Vintage,31
2,Grand Central Publishing,25
3,Penguin Classics,24
4,Ballantine Books,19
...,...,...
329,Turtleback,1
330,Atheneum Books for Young Readers: Richard Jack...,1
331,Penguin Signet,1
332,Victor Gollancz,1


### Определим автора с самой высокой средней оценкой книг, при учете только книг с 50 и более оценками

In [50]:
query = \
'''
SELECT
     author,
     AVG(sub.avg_rating) AS high_avg_rating
  FROM
     (SELECT
          author,
          books.book_id,
          AVG(rating) AS avg_rating
  FROM 
     authors
      LEFT JOIN books ON books.author_id = authors.author_id
      LEFT JOIN ratings ON ratings.book_id = books.book_id
  GROUP BY
      author,
      books.book_id
  HAVING
      COUNT(rating) > 50) AS sub
  GROUP BY
      author
  ORDER BY
      high_avg_rating DESC
'''

sql_result(query)

,author,high_avg_rating
0,J.K. Rowling/Mary GrandPré,4.283844
1,Markus Zusak/Cao Xuân Việt Khương,4.264151
2,J.R.R. Tolkien,4.258446
3,Louisa May Alcott,4.192308
4,Rick Riordan,4.080645
5,William Golding,3.901408
6,J.D. Salinger,3.825581
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,3.789474
8,William Shakespeare/Paul Werstine/Barbara A. M...,3.787879
9,Dan Brown,3.754540


### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [51]:
query = \
'''
SELECT
     AVG(review_cnt) as avg_review_cnt
  FROM
     (SELECT
     COUNT(review_id) as review_cnt
  FROM
     reviews
  WHERE username IN
      (SELECT
           username
  FROM
     ratings
  GROUP BY
     username
  HAVING
     COUNT(ratings) > 50)
  GROUP BY
     username) as sub_review_cnt 

'''
sql_result(query)

,avg_review_cnt
0,24.333333


## Выводы:

1. После 1 января 2000 года вышло 819 книг;

2. На книгу Twilight (Twilight #1) вышло максимальное количество обзоров - 7, рейтинг книги - 3.7;

3. Издательство, выпустившее наибольшее число книг толще 50 страниц: Penguin Books, они выпустили 42 книги;

4. Автор с самой высокой средней оценкой книг: J.K. Rowling/Mary GrandPré с рейтингом 4.28;

5. Среднее количество обзоров от пользователей, которые поставили больше 50 оценок: 24.3.